In [ ]:
import numpy as np
import matplotlib.pylab as plt

<h2>High-dimensional data</h2>

Imaging data often yields high-dimensional features. Most of the time the number of dimensions is much higher than the number of samples. In these cases, a few difficulties arise: 
- it is difficult to visualize classification or regresion models
- when the number of samples is lower than number of dimensions, this may lead to trivial solutions
- models need to use some sort of feature selection or regularization to overcome this challenge

In this section, we will focus on two high-dimensional problems in neuroimaging. We will use measurements extracted from brain MRI: volumes of different anatomical structures and thickness of the cortical mantle at different locations (all shown in the images below).

<img src="brain.png" alt="Brain"  style="float: left; width: 15%; margin-right: 1%; margin-bottom: 0.5em;">

<img src="aseg.png" alt="ASEG" style="float: left; width: 15%; margin-right: 1%; margin-bottom: 0.5em;">

<img src="cort_thickness.png" alt="CORT" style="float: left; width: 15%; margin-right: 1%; margin-bottom: 0.5em;">
<p style="clear: both;">

<h3>Alzheimer's Disease classification</h3>

First one is to classify subjects with Alzheimer's Disease (AD) and healthy elderly (CN) using cortical thickness maps. We will directly work with the cortical thickness values extracted from 290 individuals and aligned on the same reference frame (figure on the right shows one such example).

In [ ]:
# features are saved in a matrix
features = np.loadtxt('data/features_ad_classification.txt')
# labels are saved as a vector
labels = np.loadtxt('data/labels_ad_classification.txt')
# printing information on the dataset
print("Number of subjects: {}".format(features.shape[0]))
print("Number of features (thickness values): {}".format(features.shape[1]))
print("Number of AD cases: {}".format(np.sum(labels)))
print("Number of CN cases: {}".format(np.sum(1-labels)))

<h4> Exercise 6 </h4>
In this exercise you will use a model of your choice to build a classifier for automatically discriminating subjects with Alzheimer's disease from healthy elderly using the data we just loaded.

I suggest using one of the following algorithms but you can experiment with other algorithms as well: 
- SVM
- Random Forest Classification
- Sparse Logistic Regression (logistic regression with penalty term = "l1")

You will compute prediction error on the training set (using the entire dataset) and estimate generalization accuracy using 5-fold stratified cross-validation as we have seen in the previous section. Please use classification accuracy and you can use "accuracy_score" function we've seen earlier. 

In [ ]:
# TODO

<h3>Age Regression</h3>

In the second task we will perform age regression using volumes of different anatomical structures. The underlying idea is that as humans age changes happen in the brain. Certain structures get larger and others get smaller. 

Let us first read the dataset:

In [ ]:
# features are saved in a matrix
# note that here we read a csv file with np.loadtxt - this is another alternative to reading csv files
features = np.loadtxt('data/features_age_regression.csv', delimiter=',').T
# labels are saved as a vector
labels = np.loadtxt('data/labels_age_regression.csv', delimiter=',')
# printing information on the dataset
print("Number of subjects: {}".format(features.shape[0]))
print("Number of features: {}".format(features.shape[1]))
print("Mean age in the dataset: {}".format(np.mean(labels)))
print("Min / Max age in the dataset: {}/{}".format(np.min(labels), np.max(labels)))

<h4> Exercise 7 </h4>
In this exercise you will use a model of your choice to build a regressor for automatically predicting subjects' age from the volumes of anatomical structures, which we have read from file in the previous step. 

I suggest using one of the following algorithms but you can experiment with other algorithms as well: 
- LASSO
- Random Forest Regression
- Neural Networks

You will compute prediction error on the training set (using the entire dataset) and estimate generalization accuracy using 5-fold cross-validation <b>(not stratified!)</b> as we have seen in the previous section. Please use RMSE to compute the prediction error but you can also experiment with Pearson's correlation coefficient, which you need to find in numpy package.

In [ ]:
# TODO

<h3>Feature Importance</h3>

In the high-dimensional data we said earlier that visualization is not trivial. This is due to the high number of features. However, one may still ask a natural questions: <b>Which features were used by the algorithm to predict?</b>. Some algorithms allow for identifying such features. Most famous ones are Random Forest and Sparse regression models, such as LASSO and ElasticNet. 

In this section we will consider the problem of age regression and identify the features that the algorithm used. We will focus on the Random Forest and Lasso algorithms but a similar analysis can be performed with some others models as well. Let us first train a random forest regressor using the entire dataset. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
randfor = RandomForestRegressor(n_estimators=50)
randfor.fit(features,labels)

Next, we look at something called <b>feature importance</b> in the Random Forest algorithm. We will not go into the details of this measure and I suggest interested people to look into it. Very briefly, this measure counts the number of times a feature is used within the algorithm. Feature importance assigns an importance measure to each feature. Let us plot it 

In [ ]:
# Plotting feature importance for the random forest algorithm
plt.figure(), 
plt.stem(randfor.feature_importances_)
plt.grid('on')
plt.xlabel('Feature Indices')
plt.ylabel('Feature Importance')
plt.show()

We observe that there are three features that stand-out. 

In [ ]:
# sorting the features based on feature importance in descending order and extracting their indices
sorted_indices = np.argsort(1-randfor.feature_importances_)
print('Top 2 features with the highest importance are: {}'.format(sorted_indices[:2]))

The features Random Forest algorithm found are WM Hypointensities and Left Lateral Ventricle in the brain for those who are in neuroimaging. 

Let us now look at the LASSO algorithm and check the features it identifies as important. 

In [ ]:
from sklearn import linear_model
lasso = linear_model.Lasso()
lasso.fit(features,labels)

LASSO is a linear model where each feature gets a weight (remember the logistic regression model). The features that are important for LASSO get non-zero weights while the others get 0 weights. The non-zero weights can be positive or negative. Let us plot the weights for each features.

In [ ]:
# Plotting feature importance for the random forest algorithm
plt.figure(), 
plt.stem(lasso.coef_)
plt.grid('on')
plt.xlabel('Feature Indices')
plt.ylabel('Feature Importance')
plt.show()

We can now look at the top two features LASSO uses.

In [ ]:
# sorting the features based on feature importance in descending order and extracting their indices
sorted_indices = np.argsort(1-np.abs(lasso.coef_))
print('Top 2 features with the highest importance are: {}'.format(sorted_indices[:2]))

These features correspond to Left Thalamus and Right Putamen. 

Note that these features are not necessarily the most important and prominent changes that happens to human brain with aging. These are simply the features Random Forest and LASSO algorithms found important to predict age in the given dataset. These features may change when the dataset changes.